In [1]:
import torch
import matplotlib.pyplot as plt
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.optim as optim 
from torchsummary import summary

In [2]:
 #构建模型
class Alexnet(torch.nn.Module):
    def __init__(self):
        super(Alexnet, self).__init__()
        #第一层卷积
        #输入通道为单通道，输出通道数为96
        #使用11*11的卷积核size来捕捉对象，步长为4，填充1
        self.conv1 = torch.nn.Conv2d(1,96,kernel_size=11,stride=4,padding=1)
        #第二层卷积
        #输入通道为96，输出通道为256
        self.conv2 = torch.nn.Conv2d(96, 256, kernel_size=5, padding=2)
        #第三层卷积
        self.conv3 = torch.nn.Conv2d(256, 384, kernel_size=3, padding=1)
        #第四层卷积
        self.conv4 = torch.nn.Conv2d(384, 384, kernel_size=3, padding=1)
        #第五层卷积
        self.conv5 = torch.nn.Conv2d(384, 256, kernel_size=3, padding=1)
        #全连接第一层
        self.linear1 = torch.nn.Linear(256 * 5 * 5, 4096)
        #全连接第二层
        self.linear2 = torch.nn.Linear(4096, 4096)
        #全连接第三层
        #输出层
        self.linear3 = torch.nn.Linear(4096, 10)
        #最大池化层
        self.pooling = torch.nn.MaxPool2d(kernel_size=3, padding=1)
        #激活函数
        self.relu = torch.nn.ReLU()
        #dropout层
        self.dropout = torch.nn.Dropout(p=0.5)
        #flatten
        self.flatten = torch.nn.Flatten()
    
#前馈    
    def forward(self, x):
        x = self.pooling(self.relu(self.conv1(x)))
        x = self.pooling(self.relu(self.conv2(x)))
        x = self.relu(self.conv3(x))
        x = self.relu(self.conv4(x))
        x = self.flatten(self.pooling(self.relu(self.conv5(x))))
        x = self.dropout(self.relu(self.linear1(x)))
        x = self.dropout(self.relu(self.linear2(x)))
        x = self.relu(self.linear3(x))
        return x
model = Alexnet()

#调用显卡
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)

Alexnet(
  (conv1): Conv2d(1, 96, kernel_size=(11, 11), stride=(4, 4), padding=(1, 1))
  (conv2): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (linear1): Linear(in_features=6400, out_features=4096, bias=True)
  (linear2): Linear(in_features=4096, out_features=4096, bias=True)
  (linear3): Linear(in_features=4096, out_features=10, bias=True)
  (pooling): MaxPool2d(kernel_size=3, stride=3, padding=1, dilation=1, ceil_mode=False)
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
)

In [3]:
#交叉熵损失函数（自带softmax方法）
Loss = torch.nn.CrossEntropyLoss()
#优化器（随机梯度下降）
Optim = torch.optim.SGD(model.parameters(),lr=0.01,momentum=0.5)